# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [0]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', na_values='?')

In [0]:
df.head()

,b,30.83,0,u,g,w,v,1.25,t,t.1,01,f,g.1,00202,0.1,+
0,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
1,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
2,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
3,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+
4,b,32.08,4.000,u,g,m,v,2.50,t,f,0,t,g,360.0,0,+


In [0]:
df.isnull().sum()

b        12
30.83    12
0         0
u         6
g         6
w         9
v         9
1.25      0
t         0
t.1       0
01        0
f         0
g.1       0
00202    13
0.1       0
+         0
dtype: int64

In [0]:
df = df.dropna()

In [0]:
df.isnull().sum().sum()

0

In [0]:
df.describe()

,30.83,0,1.25,01,00202,0.1
count,652.000000,652.000000,652.000000,652.000000,652.000000,652.000000
mean,31.504847,4.836940,2.245821,2.504601,180.326687,1015.315951
std,11.847327,5.027369,3.373483,4.971962,168.423883,5257.161359
min,13.750000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22.580000,1.040000,0.165000,0.000000,72.250000,0.000000
50%,28.375000,2.855000,1.000000,0.000000,160.000000,5.000000
75%,38.250000,7.500000,2.625000,3.000000,272.000000,400.000000
max,76.750000,28.000000,28.500000,67.000000,2000.000000,100000.000000


In [0]:
# Replacing values in 'approved' columns
# from + to 1 & from - to 0 (approved and not-approved)
df = df.rename(columns={'+':'approved'})
df = df.replace({'+': 1, '-': 0})

In [0]:
df.approved.value_counts()

0    357
1    295
Name: approved, dtype: int64

In [0]:
df.shape

(652, 16)

In [0]:
df.dtypes

b            object
30.83       float64
0           float64
u            object
g            object
w            object
v            object
1.25        float64
t            object
t.1          object
01            int64
f            object
g.1          object
00202       float64
0.1           int64
approved      int64
dtype: object

In [0]:
df.columns

Index(['b', '30.83', '0', 'u', 'g', 'w', 'v', '1.25', 't', 't.1', '01', 'f',
       'g.1', '00202', '0.1', 'approved'],
      dtype='object')

In [0]:
df['01'] = df['01'].astype(float)

In [0]:
df['0.1'] = df['0.1'].astype(float)

In [0]:
# All continuous values as floats
df.dtypes

b            object
30.83       float64
0           float64
u            object
g            object
w            object
v            object
1.25        float64
t            object
t.1          object
01          float64
f            object
g.1          object
00202       float64
0.1         float64
approved      int64
dtype: object

In [0]:
df.head()

,b,30.83,0,u,g,w,v,1.25,t,t.1,01,f,g.1,00202,0.1,approved
0,a,58.67,4.460,u,g,q,h,3.04,t,t,6.0,f,g,43.0,560.0,1
1,a,24.50,0.500,u,g,q,h,1.50,t,f,0.0,f,g,280.0,824.0,1
2,b,27.83,1.540,u,g,w,v,3.75,t,t,5.0,t,g,100.0,3.0,1
3,b,20.17,5.625,u,g,w,v,1.71,t,f,0.0,f,s,120.0,0.0,1
4,b,32.08,4.000,u,g,m,v,2.50,t,f,0.0,t,g,360.0,0.0,1


## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end. This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

### Exploring Continuous Features

#### Approved

In [0]:
# DataFrame of Approved - Continuous Features
cont_app = df[df.approved == 1].select_dtypes(include=['float64'])

In [0]:
cont_app.shape

(295, 6)

In [0]:
cont_app.head()

,30.83,0,1.25,01,00202,0.1
0,58.67,4.460,3.04,6.0,43.0,560.0
1,24.50,0.500,1.50,0.0,280.0,824.0
2,27.83,1.540,3.75,5.0,100.0,3.0
3,20.17,5.625,1.71,0.0,120.0,0.0
4,32.08,4.000,2.50,0.0,360.0,0.0


#### Not Approved

In [0]:
# DataFrame of NOT Approved - Continuous Features
cont_notapp = df[df.approved == 0].select_dtypes(include=['float64'])

In [0]:
cont_notapp.shape

(357, 6)

In [0]:
cont_notapp.head()

,30.83,0,1.25,01,00202,0.1
69,32.33,7.50,1.585,0.0,420.0,0.0
71,38.58,5.00,13.500,0.0,980.0,0.0
72,44.25,0.50,10.750,0.0,400.0,0.0
73,44.83,7.00,1.625,0.0,160.0,2.0
74,20.67,5.29,0.375,1.0,160.0,0.0


#### Two-sample T-test

In [0]:
stats.ttest_ind(cont_app['30.83'], cont_notapp['30.83'])

Ttest_indResult(statistic=4.679127797644734, pvalue=3.5069404140554282e-06)

In [0]:
# Greatest statistical difference: Feature '01' (T-statistic of 11.36)
# Lowest statistical difference: Feature '00202' (T-statistic of -2.18 & pvalue closest to 0.05)
cols = cont_app.columns
for col in cols:
    stat, pvalue = stats.ttest_ind(cont_app[col], cont_notapp[col])
    print(col, '--->', stat, '     ', pvalue)

30.83 ---> 4.679127797644734       3.5069404140554282e-06
0 ---> 5.450090587799932       7.154104218580255e-08
1.25 ---> 9.020662467924037       2.0835108311601946e-18
01 ---> 11.358835276384589       2.1156266716722337e-27
00202 ---> -2.1881972892346777       0.029010481356813344
0.1 ---> 4.485021540265138       8.619679105602507e-06


### Exploring Categorical Features

#### Chi-square Test Using Crosstab Table

In [0]:
categories = ['b', 'u', 'g', 'w', 'v', 't', 't.1', 'f', 'g.1']
for category in categories:
    ct = pd.crosstab(df.approved, df[category])
    chi = stats.chi2_contingency(ct)
    print(ct)
    print('\n', '--->', chi, '\n')

b           a    b
approved          
0         108  249
1          95  200

 ---> (0.20305092453739787, 0.652269416331663, 1, array([[111.15184049, 245.84815951],
       [ 91.84815951, 203.15184049]])) 

u         l    u    y
approved             
0         0  250  107
1         2  248   45

 ---> (21.597091845920502, 2.042918743690437e-05, 2, array([[  1.09509202, 272.67791411,  83.22699387],
       [  0.90490798, 225.32208589,  68.77300613]])) 

g           g  gg    p
approved              
0         250   0  107
1         248   2   45

 ---> (21.597091845920502, 2.042918743690437e-05, 2, array([[272.67791411,   1.09509202,  83.22699387],
       [225.32208589,   0.90490798,  68.77300613]])) 

w         aa   c  cc   d   e  ff   i  j   k   m   q  r   w   x
approved                                                      
0         33  73  11  19  10  43  41  7  35  22  26  1  30   6
1         19  60  29   7  14   7  14  3  13  16  49  2  32  30

 ---> (89.51520073703931, 1.73000426794241

## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

*Your words here!*

### T-tests

Performed two-sample t-test on each continuous feature comparing the means of _approved_ and _not approved_. __There seems to be _statistical difference_ in all the compared features__.

Greatest statistical difference is seen in _feature '01'_. With t-statistic of 11.358835276384589 & pvalue of 2.1156266716722337e-27.

Lowest statistical difference is seen in _feature '00202'_. With t-statistic of -2.18 & pvalue closest to 0.05.

### Chi-square tests
Performed Chi-square test of independence of variables based on crosstab table for each categorical feature based on approval. __There seems to be _statistical correlation (dependance)_ in MOST compared features__.

Features _b_, _f_, seem to show correlation between those approved and not approved. Both have pvalues.

After all, the questions, are these observations _practically relevant_. It would take knowing what each feature means in this case.

### Most Challenging part
Know _how_ to apply the chi-square test seemed most difficult part of the challenge. I'm not completely sure if I have applied it correctly to each feature.